In [13]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

/kaggle/input/indore-city-house-price-predection/IndoreHP_Sample.csv
/kaggle/input/indore-city-house-price-predection/IndoreHP_Test.csv
/kaggle/input/indore-city-house-price-predection/IndoreHP_Train.csv


In [14]:
import pandas as pd

train = pd.read_csv('../input/indore-city-house-price-predection/IndoreHP_Train.csv')
test = pd.read_csv('../input/indore-city-house-price-predection/IndoreHP_Test.csv')

print(train.info())
print(train.duplicated().sum())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 350 entries, 0 to 349
Data columns (total 15 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   ID                           350 non-null    int64  
 1   CRIM                         350 non-null    float64
 2   ZN                           350 non-null    float64
 3   INDUS                        350 non-null    float64
 4   RIVER_FLG                    350 non-null    int64  
 5   nitric oxides concentration  350 non-null    float64
 6   #rooms/dwelling              350 non-null    float64
 7   AGE                          350 non-null    float64
 8   DIS                          350 non-null    float64
 9   RAD                          350 non-null    int64  
 10  TAX                          350 non-null    int64  
 11  PTRATIO                      350 non-null    float64
 12  B                            350 non-null    float64
 13  LSTAT               

In [15]:
X=train.drop(columns=['MEDV'])
y=train['MEDV']

In [16]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


from sklearn.preprocessing import MinMaxScaler

# Perform Min-Max scaling on the features
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
test = scaler.transform(test)


from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor()

from skopt import BayesSearchCV
param_space = {
    'n_estimators': (10, 100),
    'max_depth': (3, 10),
    'min_samples_split': (2, 10),
    'min_samples_leaf': (1, 5),
    'max_features': (0.1, 1.0)
}
opt = BayesSearchCV(model, param_space, n_iter=20, cv=5, scoring='neg_mean_squared_error', random_state=42)

In [17]:
# Wrap BayesSearchCV with tqdm
from tqdm import tqdm
with tqdm(total=opt.total_iterations, desc='Optimizing') as pbar:
    def callback(res):
        pbar.update(1)

    opt.fit(X_train, y_train, callback=callback)

Optimizing:  20%|██        | 20/100 [00:25<01:41,  1.26s/it]


In [18]:
# Get the best hyperparameters found by BayesSearchCV
best_params = opt.best_params_
print("Best Hyperparameters:", best_params)

from sklearn.metrics import mean_absolute_error
# Evaluate the model with the best hyperparameters on the validation set
best_model = opt.best_estimator_
y_pred = best_model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print("Validation MAE:", mae)

Best Hyperparameters: OrderedDict([('max_depth', 7), ('max_features', 0.5379844709434165), ('min_samples_leaf', 1), ('min_samples_split', 9), ('n_estimators', 72)])
Validation MAE: 2.465559530731797


In [19]:
y_pred1=best_model.predict(test)

submission=pd.read_csv('../input/indore-city-house-price-predection/IndoreHP_Sample.csv')
submission['MEDV']=y_pred1
submission.to_csv('./submission2.csv', index=False)
submission.head()

,ID,MEDV
0,400,20.753930
1,401,26.445774
2,402,31.167605
3,403,21.331910
4,404,28.948015
